# Compositional analysis of the ACM cardiac leucocytes 6mpi dataset (Pkp2+Ttn) - all celltypes - using scCoda `pertpy`:
- **Developed by**: Christian Eger
- **Modified by**: Alexandra Cirnu
- **Talavera-López Lab - Würzburg Institute for Systems Immunology - Faculty of Medicine - Julius Maximilian Universität Würzburg**
- **Created on**: 231214
- **Last modified**: 240227

#### Environment: pertpy

In [3]:
import warnings

import pandas as pd

warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")

import numpy as np
            #import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import altair as alt

import scanpy as sc

import pertpy as pt
from statannot import add_stat_annotation
from scipy import stats


In [ ]:
sc.logging.print_versions()

Manual git installation of pertpy necessary due to Apple silicon
$ git clone https://github.com/theislab/pertpy.git
$ cd pertpy
$ pip install .

In [ ]:
adata = sc.read_h5ad("Marburg_cell_states_locked_scANVI_ctl230901.raw.h5ad")
print(adata)
#adata object created from the annotated h5ad file.

In [ ]:
adata.obs

In [ ]:
#adata.obs["infectionANDdisease"] = adata.obs["infection"].str.cat(adata.obs["disease"], sep = "&")
adata.obs
adata.obs["group"].cat.categories
#creating a new obs based on adding obs infection and obs disease --> CTRL&COPD, IAV&COPD, CTRL&healthy, IAV&healthy

In [ ]:
adata.obs["cell_compartment"]
#adata.obs["cell_compartment"].cat.categories


In [ ]:
adata.obs["cell_states"].cat.categories

create notebook for every compartment and compare across cell states
visualization improvement
reference to ctrl healthy

In [ ]:
sccoda_model = pt.tl.Sccoda()
sccoda_data = sccoda_model.load(
    adata,
    type="cell_level",
    generate_sample_level=True,
    cell_type_identifier="cell_compartment",
    sample_identifier="batch",
    covariate_obs=["infection", "disease", "group"],
)
sccoda_data

In [ ]:
pt.pl.coda.boxplots(
    sccoda_data,
    modality_key="coda",
    feature_name="group",
    y_scale="count",
    figsize=(12, 5),
    add_dots=True,
    args_swarmplot={"palette": ["red"]},
)
plt.show()

In [ ]:
pt.pl.coda.stacked_barplot(
    sccoda_data, modality_key="coda", feature_name="group", figsize=(4, 2)
)
plt.show()

## 11.12.23

Function sccoda.run_nuts() broken at the moment
https://github.com/theislab/pertpy/issues/454
Downgrading Jax didn't solve the issue for me.

--> solution: completely new environment, install jax=0.4.20 prior to installing pertpy

In [ ]:
sccoda_data = sccoda_model.prepare(
    sccoda_data,
    modality_key="coda",
    formula="infection",
    reference_cell_type="automatic",
)
sccoda_model.run_nuts(sccoda_data, modality_key="coda", rng_key=1234)

In [ ]:
sccoda_data

In [ ]:
sccoda_model.set_fdr(sccoda_data, 0.2)

In [ ]:
sccoda_model.credible_effects(sccoda_data, modality_key="coda")

In [ ]:
sccoda_data["coda"].varm["effect_df_infection[T.IAV]"]

In [ ]:
sns.barplot(data=sccoda_data["coda"].varm["effect_df_infection[T.IAV]"], x="Cell Type", y="log2-fold change", hue="Cell Type")

In [ ]:
pt.pl.coda.effects_barplot(sccoda_data, "coda", "disease")
plt.show()